<a href="https://colab.research.google.com/github/farhanadhilah/Exploratory-Data-Analysis/blob/main/Crawling%20Pertamax%20Tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Social Media Analytics (Twitter)**


Nadhilah Farhana (2106779516)


# **Data Crawling**

In [ ]:
import warnings; warnings.simplefilter('ignore')
import nltk

try:
    import google.colab
    IN_COLAB = True
    !wget https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/taudataDDGsna.py
    !mkdir data
    !wget -P data/ https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/data/contoh.pdf
    !wget -P data/ https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/data/slang.txt
    !wget -P data/ https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/data/stopwords_id.txt
    !wget -P data/ https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/data/stopwords_en.txt
    !wget -P data/ https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/data/kata_dasar.txt
    !wget -P data/ https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/data/wn-ind-def.tab
    !wget -P data/ https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/data/wn-msa-all.tab
    !wget -P data/ https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/data/ind_SA.csv
    !wget -P data/ https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/data/all_indo_man_tag_corpus_model.crf.tagger

    !pip install --upgrade spacy python-crfsuite unidecode textblob sastrawi sklearn-pycrfsuite
    !pip install --upgrade unidecode twython tweepy beautifulsoup4 tika
    !python -m spacy download en
    !python -m spacy download xx
    !python -m spacy download en_core_web_sm
    nltk.download('popular')
except:
    IN_COLAB = False
    print("Running the code locally, please make sure all the python module versions agree with colab environment and all data/assets downloaded")

--2023-04-19 14:27:35--  https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/taudataDDGsna.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8030 (7.8K) [text/plain]
Saving to: ‘taudataDDGsna.py.1’

taudataDDGsna.py.1  100%[===================>]   7.84K  --.-KB/s    in 0s      

2023-04-19 14:27:35 (68.9 MB/s) - ‘taudataDDGsna.py.1’ saved [8030/8030]

mkdir: cannot create directory ‘data’: File exists
--2023-04-19 14:27:35--  https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/data/contoh.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP reque

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to-date!
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Package names is already up-to-date!
[nltk_data]    | Do

In [ ]:
import taudataDDGsna as tau
import tweepy, json, urllib.request, requests
from urllib.request import Request, urlopen
from twython import TwythonStreamer
from bs4 import BeautifulSoup as bs
from tqdm import tqdm
"Done"

'Done'

<h2 id="Aturan-twitter">Aturan, bentuk data, &amp; error codes twitter</h2>

<ol>
	<li>
	<p><a href="https://dev.twitter.com/rest/public/rate-limiting" target="_blank">https://</a><a href="https://dev.twitter.com/rest/public/rate-limiting" target="_blank">dev.twitter.com/rest/public/rate-limiting</a></p>
	</li>
	<li>
	<p><a href="https://dev.twitter.com/overview/terms/agreement-and-policy" target="_blank">https://dev.twitter.com/overview/terms/agreement-and-policy</a></p>
	</li>
	<li>
	<p><a href="https://dev.twitter.com/overview/api/response-codes" target="_blank">https://</a><a href="https://dev.twitter.com/overview/api/response-codes" target="_blank">dev.twitter.com/overview/api/response-codes</a></p>
	</li>
	<li>
	<p><a href="https://dev.twitter.com/overview/api/tweets" target="_blank">https://</a><a href="https://dev.twitter.com/overview/api/tweets" target="_blank">dev.twitter.com/overview/api/tweets</a></p>
	</li>
</ol>

In [ ]:
# Contoh API Keys (Sesuaikan dengan API keys masing-masing)
Ck = 'p76aa3SQdGXG6KlOFB2rlEYN8' # consumer_key
Cs = '7QY41F7LK449lKVUoi1sKlZidq6ALYVh4FEKQj2Ph6XmT4IN7N' # consumer_secret
At = '529202222-MB1NLjJGxeZA6gKiiDFaHrPMVdGid1oOp7ZwDXyL' # access_token
As = '1Yali9pzlbHyMRq5ta5FAPu1KQ4r2UsS6ujZGfXbNQxFo' # access_secret

'Done'

'Done'

In [ ]:
import tweepy

auth = tweepy.auth.OAuthHandler(Ck, Cs)
auth.set_access_token(At, As)
api = tweepy.API(auth, wait_on_rate_limit=True, timeout=180, retry_count=5, retry_delay=3)
usr_ = api.verify_credentials()
print('Welcome "{}" you are now connected to twitter server'.format(usr_.name))

Welcome "Nadila" you are now connected to twitter server


In [ ]:
# Kita bisa post, delete, follow, like, reply, retweet, dsb.
# WARNING ... Status harus dibedakan, jika tidak akan gagal karena twitter mendeteksi status duplikat.

status = "Happy me"
t = api.update_status(status)
# Silahkan cek timeline setelah ini
"Done! ... Silahkan cek timeline twitter anda."

'Done! ... Silahkan cek timeline twitter anda.'

In [ ]:
# https://www.latlong.net/convert-address-to-lat-long.html

alamat = "Seoul, South Korea"
lat = '37.532600'
lon = '127.024612'

radius = '10000km'
Geo = ','.join([lat, lon, radius])

qry = '"Me Too movement korean" OR "me too movement in south korea'
#qry = '"feminist korea" OR "anti feminist korea"'
N = 100 # jumlah N tweet "terkini" yang ingin diambil
T = []
for post in tqdm(tweepy.Cursor(api.search_tweets, q=qry, lang='en', tweet_mode='extended', since_id='2018-01-01').items(N)):
    T.append(post)

"Done"

0it [00:00, ?it/s]


'Done'

In [ ]:
T[0]._json

IndexError: ignored

In [ ]:
# T bukan sekedar list of tweets
tweet = [t._json for t in T]
print('Berhasil mendapatkan {} tweets'.format(len(tweet)))

Berhasil mendapatkan 11 tweets


In [ ]:
# Data pertama
tweet[0]

{'contributors': None,
 'coordinates': None,
 'created_at': 'Sat Jun 11 02:35:48 +0000 2022',
 'display_text_range': [0, 140],
 'entities': {'hashtags': [],
  'symbols': [],
  'urls': [],
  'user_mentions': [{'id': 1215268532532834304,
    'id_str': '1215268532532834304',
    'indices': [3, 15],
    'name': 'Balvyah',
    'screen_name': 'BalvyHaddad'},
   {'id': 1516416251572244480,
    'id_str': '1516416251572244480',
    'indices': [17, 31],
    'name': '🙏🙏🙏',
    'screen_name': 'A1234567890ZQ'},
   {'id': 1524694713651015680,
    'id_str': '1524694713651015680',
    'indices': [32, 48],
    'name': '🇮🇩🇵🇸JOHAN-ISWAHYUDI🇵🇸🇮🇩',
    'screen_name': 'JohanIswahyud10'},
   {'id': 2866140311,
    'id_str': '2866140311',
    'indices': [49, 60],
    'name': 'Bhisma Yang Agung',
    'screen_name': 'MahaBhisma'},
   {'id': 1479144195721863170,
    'id_str': '1479144195721863170',
    'indices': [61, 75],
    'name': 'Maul',
    'screen_name': 'UnitedDay2022'}]},
 'favorite_count': 0,
 'favorit

In [ ]:
# Datanya berbentuk JSON
tweet[10]['created_at'], tweet[10]['full_text']

('Tue Apr 11 18:52:57 +0000 2023',
 'Ye T-Series wala haii ?\n\nIska bhi #Metoo me naam aaya tha...\n\nAb BJP ki bhakti me bjp walo k album launch krta !! https://t.co/DQp6HSKvcM')

In [ ]:
# Contoh mengakses data spesifik pada tweet yang pertama:
print('tweet pertama oleh "{}" : "{}"'.format(tweet[0]['user']['screen_name'],tweet[0]['full_text']))

tweet pertama oleh "Ciput84725571" : "RT @SmashNewsCorp: Smash News - Industri plat merah belum menjual bahan bakar minyak( BBM) non- subsidi yang cocok dengan harga keekonomian…"


# <center><font color="blue">Menyimpan hasil crawling (sederhana ke Text file berformat json)</font></center>

In [ ]:
import json
def saveTweets(tweets, file='Tweets.json'): #in Json Format
    with open(file, 'w') as f:
        for t in tweets:
            try:
                f.write(json.dumps(t)+'\n')
            except:
                pass

In [ ]:
# Menyimpan hasil crawling twitter
fileName = 'Tweets_2022.json'
saveTweets(tweet, file=fileName)
print('Saved to '+fileName)

Saved to Tweets_2022.json


# <center><font color="blue">Load Kembali?</font></center>

In [ ]:
def loadTweets(file='Tweets.json'):
    f=open(file,encoding='utf-8', errors ='ignore', mode='r')
    T=f.readlines();f.close()
    for i,t in enumerate(T):
        T[i] = json.loads(t.strip())
    return T

In [ ]:
# Me-load kembali jika (misal) analisa ingin dilakukan di lain waktu
# Sengaja nama variabelnya saya bedakan (T2)
T2 = loadTweets(file=fileName)
print('tweet pertama oleh "{}" : "{}"'.format(T2[0]['user']['screen_name'],T2[0]['full_text']))

tweet pertama oleh "Ciput84725571" : "RT @SmashNewsCorp: Smash News - Industri plat merah belum menjual bahan bakar minyak( BBM) non- subsidi yang cocok dengan harga keekonomian…"


In [ ]:
# Contoh mengambil hanya data tweet
D = [t['full_text'] for t in T2]
D[:5] # 5 tweet pertama

['RT @SmashNewsCorp: Smash News - Industri plat merah belum menjual bahan bakar minyak( BBM) non- subsidi yang cocok dengan harga keekonomian…',
 '@needtoplove DM aja',
 '@mlbbfess Ngisinya pertamax plus dia bukan premium',
 'Ujan2 enak kali ya, main bertiga...\n Lagi di reddoors nih, siapa tau ada yang mau joint.. \n\n#gaysemarang',
 '@UNSfess_ Mau diisi apa kak? Pertalite apa pertamax. Kalau Pertamax skrg lagi mahal.']

In [ ]:
# Mari kita coba #1
topic = 'from:jokowi'
T = api.search_tweets(q=topic, lang='id', tweet_mode = 'extended')
tweet = [t._json for t in T]
isiTweet = [t['full_text'] for t in tweet]
isiTweet

['Setiap kita punya peran dalam pelestarian alam. Setiap orang bisa punya andil menyelamatkan bumi.\n\nSelamat Hari Lingkungan Hidup Sedunia https://t.co/bbhaJO72tw',
 'Balap Formula E ini adalah tontonan yang semakin digemari di masa depan mengingat kian banyaknya pemakai mobil listrik. Ajang balap ini juga bagus untuk Indonesia yang akan membangun ekosistem kendaraan listrik. https://t.co/bOq9W7vQxl',
 'Hari ini, kita menyaksikan langsung atau melalui tayangan televisi, gelaran balap 2022 Jakarta E-Prix di Sirkuit Formula E, Jakarta. Alhamdulillah semuanya berjalan lancar. \n\nSelamat kepada para pemenang. Semoga ke depan, lebih banyak lagi penyelenggaraan balap serupa Formula E. https://t.co/qUgVWwk5rX',
 'Semua milik Allah SWT dan hanya kepadaNya kita akan kembali. \n\nTurut berbelasungkawa atas berpulangnya ananda Emmeril Kahn Mumtadz, semoga seluruh keluarga Bapak Ridwan Kamil diberikan ketabahan dan keikhlasan. https://t.co/lOeBmciGQj',
 'Menonton pacuan kuda yang seru di lapang

In [ ]:
# https://www.latlong.net/convert-address-to-lat-long.html

alamat = "Graha Cipta 5, Jejalen Jaya, Tambun Utara, Bekasi"
lat = '-6.236690'
lon = '107.063560'

"Done"

'Done'

In [ ]:
# Mari kita coba #3 gunakan google (map) untuk koordinat suatu lokasi
# http://thoughtfaucet.com/search-twitter-by-location/
# misal search tweet tentang "makanan" di Depok dan sekitarnya
radius = '10km'
Geo = ','.join([lat, lon, radius])
qry = 'seblak'
for tweet_ in tweepy.Cursor(api.search_tweets, q=qry, geocode=Geo).items(N):
    print([tweet_.created_at, tweet_.text.encode('utf-8'), tweet_.user.id, tweet_.geo])

[datetime.datetime(2022, 6, 5, 10, 53, 1, tzinfo=datetime.timezone.utc), b'@Aldoaria @FOOD_FESS Seblak didieu ewh ngeunah2 na acan beda jeung jieunan indung sorangan', 747725422339698688, None]
[datetime.datetime(2022, 6, 5, 4, 40, 28, tzinfo=datetime.timezone.utc), b'Mie seblak ajalah yak https://t.co/bp11PYZeSF', 941497184289095680, None]
[datetime.datetime(2022, 6, 4, 17, 47, 40, tzinfo=datetime.timezone.utc), b'@fosilmikro mereka uangnya habis beli seblak ketimbang deodorant. Padahal deodorant sama serbuk tawas buat mandi murah lho.', 1361422247978704896, None]
[datetime.datetime(2022, 6, 4, 15, 42, 28, tzinfo=datetime.timezone.utc), b'begah bgt abis makan seblak 2 mangkok \xf0\x9f\x98\xac', 1214485917307588608, None]
[datetime.datetime(2022, 6, 4, 14, 50, 43, tzinfo=datetime.timezone.utc), b'Seblak kasih nasi juga belum mempan #kenyangnya \xf0\x9f\x98\xb9\xf0\x9f\x98\x8c', 130684283, None]
[datetime.datetime(2022, 6, 4, 13, 24, 58, tzinfo=datetime.timezone.utc), b'cowok kok ga per

In [ ]:
# Streaming tweets. Untuk percobaan pilih topicS sesuatu yg sedang trending/populer "saat ini".
# Atau bisa coba dengan mengirim tweet sendiri :)
from twython import TwythonStreamer

def streamTwitter(topicS, lang):
    class MyStreamer(TwythonStreamer):
        def on_success(self, data):
            global count
            count+=1
            print('tweet from {}, post: {}'.format(data['user']['screen_name'], data['text']))
            if count==maxTweet:
                print('\nFinished streaming %.0f tweets' %(maxTweet)); self.disconnect()
        def on_error(self, status_code, data):
            print('Error Status = %s' %status_code); self.disconnect()

    while count<maxTweet:
        stream = MyStreamer(Ck, Cs, At, As)
        stream.statuses.filter(track=topicS)

In [ ]:
maxTweet, count = 5, 0 # Rubah sesuai dengan kebutuhan, Untuk percobaan ini cukup (misal) 12 tweet
lan = set(['en','id']) # bahasa bisa dipilih > 1
topicS = ['eril', 'ridwan kamil'] # Bisa>1

streamTwitter(topicS, lan)

tweet from ByunnBaekhyun23, post: RT @NCT139FESS: Arkana sayang bgt ma a eril ,foto a eril di dicium trs😭😭😭

Lihat video ini nangis bgt Al Fatihah buat a eril 🙏🏻 https://t.c…
tweet from GeorginaZaskia, post: RT @nickspegal9yaho: Rest in love 
Tiba dirumah dinas gubernur jawa Barat ridwan kamil langsung disambut arkana adik angkat nya eril  , sed…
tweet from lailiya25829261, post: RT @DawnOfMancs: Jepretan tadi siang, Adek Arkana lagi liat foto Aa Eril yg Gagah. Sedih bgt tapi kita harus ikhlas.

Pamit pulang ke JKT y…
tweet from lestariology, post: RT @AREAJULID: Dis! Nyesek banget liat ini, anak sekecil Arka pasti belum ngerti kalo kakaknya udah men1nggal😭😭

Rest in love, Eril. Al fat…
tweet from like_limelightt, post: RT @AREAJULID: Dis! Nyesek banget liat ini, anak sekecil Arka pasti belum ngerti kalo kakaknya udah men1nggal😭😭

Rest in love, Eril. Al fat…

Finished streaming 5 tweets


In [ ]:
# preprocess "D": kumpulan tweet
slangFixId = tau.loadCorpus(file = 'data/slang.txt', sep=':')
stopId, _ = tau.LoadStopWords(lang='id')
stopId.add("rt")

list(stopId)[:5]

['diketahuinya', 'menuturkan', 'seolah-olah', 'diakhirinya', 'kalaupun']

In [ ]:
from tqdm import tqdm

cleanD = []
for t in tqdm(D):
    doc = tau.cleanText(t, fix=slangFixId, lan='id', stops = stopId)
    cleanD.append(doc)

"Done"

100%|██████████| 1000/1000 [00:00<00:00, 1719.43it/s]


'Done'

In [ ]:
print(D[0], end='\n\n')
print(cleanD[0], end='\n\n')

RT @SmashNewsCorp: Smash News - Industri plat merah belum menjual bahan bakar minyak( BBM) non- subsidi yang cocok dengan harga keekonomian…

smashnewscorp smash news industri plat merah menjual bahan bakar minyak bbm non subsidi cocok harga keekonomian



In [ ]:
# Save ke txt ... lalu nanti akan di upload ke Voyant Tools
filename = 'tweets_2021.txt'
with open(filename, 'w') as f:
    for T in cleanD:
        f.write(T+'\n')

"Tweets Saved!"

# <center><font color="blue">HashTag Analysis</font></center>

* Frequency-Based
* Bisa ditambahkan analisa hashtags analysis ini dari waktu ke waktu

In [ ]:
import re

HTfilters = set(['zz', 'architec', 'prize', 'stirli', 'architect', 'london', 'cpd', 'design', 'stirling', 'photogr', 'gemini',
                 'mule', 'karaoke', 'playing', 'official', 'berita', 'follow', 'retweet', 'mufc', 'ntms', 'infolimit', 'eeaa',
                 'eaa', 'cfc', 'caprico', 'breaking','news', 'libra', 'mereka', 'brankas', 'psikolog', 'aquarius', 'klc'])
# modifikasi HTfilters sesuai data kamu
HT = {'hashtags':[]}
count = 0
getHashTags = re.compile(r"#(\w+)")
for i, d in tqdm(enumerate(D)):
    hashtags = re.findall(getHashTags, d)
    if hashtags:
        TG = []
        for tag in hashtags:
            dTag = str(tag).strip().lower()
            if len(dTag)>2:
                add = True
                for f in HTfilters:
                    if f in dTag:
                        add=False; break
                if add:
                    TG.append('#'+dTag); count += 1
            HT['hashtags'].append(TG)
print('Found {} number of hashtags'.format(count))

In [ ]:
import pandas as pd

N = 50 # Number of top frequent hashtags to be plotted

dtHT = [x for t in tqdm(HT['hashtags']) for x in t] # any(h not in x for h in HTfilters)
dtHT = pd.Series(dtHT)
dtHT = dtHT.value_counts()
dtHT = dtHT.sort_index()
dtHT = dtHT.sort_values(ascending = False)
dtHT.to_csv('hashTags_2021.csv', encoding='utf8')
dtHT = dtHT.iloc[:N]

p = dtHT.plot(kind='barh', figsize=(12,8), legend = False)

# Web Scrapping?

In [ ]:
import urllib.request
from bs4 import BeautifulSoup as bs

URL = 'https://www.beritasatu.com/beritasatu-tv'
Doc = urllib.request.urlopen(URL).read()
Doc = bs(Doc,'lxml').text
print(Doc)

# Loading Local Documents

1. Instalasi Java
 - JDK 8 ... ingat harus JDK 8
 - https://www.oracle.com/id/java/technologies/javase/javase-jdk8-downloads.html
 - Set Java Home Directory
2. Instalasi Tika Server :
 - Download Tika App Executable Java Jar: https://archive.apache.org/dist/tika/tika-app-1.24.1.jar
 - Put in Python home directory
3. Instalasi Module Python Tika
4. Reading pdf Files (Python Code below)

### Tika can read Pdf, DocX, PPTX, etc.

In [ ]:
from tika import parser#, unpack

def readDocs(file):
    if 'pdf' in file:
        headers = {'X-Tika-PDFextractInlineImages': 'true',}
        raw = parser.from_file(file, headers=headers)
    else:
        raw = parser.from_file(file)
    if 'content' in raw.keys():
        return raw['content']
    else:
        return None

In [ ]:
try:
    doc = readDocs('data/contoh.pdf')
except:
    !mkdir data
    !wget -P data/ https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/data/contoh.pdf
    doc = readDocs('data/contoh.pdf')

print(doc)